<a href="https://colab.research.google.com/github/mgnarag/machine-learning/blob/main/Testing_and_metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/CS284/Project/"

'107.1 syllabus 1SAY1920 FINAL.gdoc'
'1C-01 Narag et al (1).pdf'
'1C-01 Narag et al.pdf'
'1FA971504AC646559D05A8ED6341AFB3[1025340].png'
'1H-04 Narag et al.pdf'
 20220802_182749.heic
 20220802_182831.heic
 20220802_183259.heic
 20220803_125819.heic
 20220803_125842.heic
 20220803_144229.heic
 20220803_144422.heic
 20220803_160724.heic
 20220803_160751.heic
 20220803_165240.heic
 20220803_165318.heic
 20220803_174701.heic
 20220803_174706.heic
 20220804_175217.heic
 20220804_175443.heic
 20220804_175616.heic
 20220804_181917.heic
 20220804_182007.heic
 20220804_182117.heic
 2S22-23_LE3-Regular-Set-A-FINAL-KEY.pdf
'A4_Physics 71 TWHFU-3_gradesheet (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades (1).pdf'
'A4_Physics 71 TWHFU-3_officialgrades.gdoc'
'Abstract - NARAG, Mark Jeremy.pdf'
'Annex 1 (copy of Journal).pdf'
'Annex 2 (Proof of employments).pdf'
'App Physics 181 THY-FX-2_studentcontactlist.xlsx'
 Architectural_designs
'Arki abstract.pdf'
'Arki full paper.pdf'
'Binarizing architectural

In [24]:
import time
import pandas as pd


import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


from PIL import Image, ImageOps
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy.ndimage import zoom

import cv2

In [ ]:
from tensorflow import keras
from PIL import Image, ImageOps
import numpy as np
import time

def crop(im,n_size):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


filename = str("[GT] dirty.png")
test = Image.open(file_path+ "Testing nbs/" + filename)
test = test.convert('RGB')
w_dirty, h_dirty = test.size


architecture = ['autoencoder_','unet_','unet_maxpool']
size = ['32_', '64_']
section = ['whole','portion']

models = []
for m in range(0,2):
  for s in range(0,2):
    for c in range(0,2):
      models.append(str(architecture[m]+size[s]+section[c]))

METRIC = []


for mod in range(0,8):
  if mod in [0, 1, 4, 5]:
    n_size = 32
    print(n_size)
  if mod in [2, 3, 6, 7]:
    n_size = 64
    print(n_size)
  start = time.time()
  model = keras.models.load_model(file_path + models[mod])
  print(model)
  xx = int(w_dirty/n_size)
  final=[]

  for portion in range(0,xx):
      im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
      w1, h1 = im1.size
      w = int(w1/n_size)
      h = int(h1/n_size)

      neverbeforeseen = np.array(crop(im1,n_size))
      #encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
      decoded_imgs = model.predict(neverbeforeseen)

      col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
      for i in range(2,h):
          col = np.vstack((col,decoded_imgs[i]))

      #y = np.where(col > 0.5,1,0) #round the values
      y= col
      y = (y * 255).astype('uint8')
      if portion == 0:
          final = y
      if portion > 0:
          final = np.hstack((final,y))

  final = np.squeeze(final)
  reconstructed = Image.fromarray(final)
  reconstructed.save(file_path + "Testing nbs/"+str(models[mod])+"_cleaned.png")

  end = time.time()
  print("The time of execution of above program is :",
        (end-start), "seconds")

METRIC.append([models[mod],psnr,ssim])
METRIC = pd.DataFrame(METRIC)
headers = ['Model','PSNR','SSIM' ]
METRIC.columns = headers
METRIC.to_csv(file_path + 'Metrics.csv')
print(METRIC)

In [82]:
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np

def calculate_ssim(img1, img2):
    # Calculate SSIM for each channel
    ssim_r = ssim(img1[:, :, 0], img2[:, :, 0],win_size=1025)
    ssim_g = ssim(img1[:, :, 1], img2[:, :, 1],win_size=1025)
    ssim_b = ssim(img1[:, :, 2], img2[:, :, 2],win_size=1025)

    # Take the average of SSIM values for all channels
    ssim_index = (ssim_r + ssim_g + ssim_b) / 3.0

    return ssim_index

def calculate_psnr(img1, img2):
    # Calculate PSNR for each channel
    psnr_r = cv2.PSNR(img1[:, :, 0], img2[:, :, 0])
    psnr_g = cv2.PSNR(img1[:, :, 1], img2[:, :, 1])
    psnr_b = cv2.PSNR(img1[:, :, 2], img2[:, :, 2])

    # Take the average of PSNR values for all channels
    psnr_value = (psnr_r + psnr_g + psnr_b) / 3.0

    return psnr_value


# Load your colored images
image1 = cv2.imread(file_path+ "Testing nbs/[GT] clean.png")

METRIC = []
for mod in range(0,8):
  image2 = cv2.imread(file_path + "Testing nbs/"+str(models[mod])+"_cleaned.png")



  # Calculate PSNR
  psnr = calculate_psnr(image1, image2)
  print(psnr)

  # Calculate SSIM
  #ssim_ = calculate_ssim(image1, image2)
  ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)
  print(ssim_)

  METRIC.append([models[mod],psnr,ssim])


METRIC = pd.DataFrame(METRIC)
headers = ['Model','PSNR','SSIM' ]
METRIC.columns = headers
METRIC.to_csv(file_path + 'Metrics.csv')
#print(METRIC)

23.155282353084456


<ipython-input-82-a1a7cf8751cf>:43: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)


0.6966021344265436
23.253675925451677
0.696490498508676
23.837538443801535
0.7020716374844901
23.583053696069328
0.698764546038596
22.01506407445719
0.6420976916607254
22.01491481095044
0.6625992167144554
22.469356522736703
0.6654117527357521
22.547820932927237
0.6717284780302376


In [86]:
ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)

print(f'SSIM Index: {ssim_}')

<ipython-input-86-5d96e6a81854>:1: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_ = ssim(image1, image2, win_size=1025,  multichannel=True)


SSIM Index: 0.6717284780302376
